In [1]:
from pathlib import Path
import sys

# notebooks/ から見てプロジェクトルートを追加
ROOT = Path.cwd().resolve().parents[0]  # notebooks の1つ上 = ~/work/keiba-ai
sys.path.append(str(ROOT))

In [2]:
from src.data.loaders.result_loader import load_results
from src.data.loaders.horse_loader import load_horse_results
from src.data.loaders.race_info_loader import load_race_info

df_result = load_results("../data/rawdf/result/result_*.csv")
df_horse = load_horse_results("../data/rawdf/horse/*.csv")
df_race_info = load_race_info("../data/rawdf/race_info/*.csv")

/home/naga2/work/keiba-ai/src/data/loaders/horse_loader.py:13: DtypeWarning: Columns (0: 着順) have mixed types. Specify dtype option on import or set low_memory=False.
  raw = pd.read_csv(p, sep=",")


In [3]:
from src.data.pipelines.build_train_table import build_train_table

df_train = build_train_table(df_result, df_race_info, df_horse)
df_train

,race_id,horse_id,race_date,place,race_type,around,course_len,weather,ground_state,race_class,...,impost,weight,weight_diff,jockey_id,trainer_id,owner_id,popularity,tansho_odds,hr_rank_mean_3,rank
0,202101010101,2018104780,2021-06-12,0,0,0.0,1200,2,0,-3.0,...,56.0,436,0,1144,1136,414030,6,10.4,5.000000,15
1,202101010101,2018100570,2021-06-12,0,0,0.0,1200,2,0,-3.0,...,54.0,456,4,666,1039,728008,1,3.7,5.666667,6
2,202101010101,2018101518,2021-06-12,0,0,0.0,1200,2,0,-3.0,...,52.0,388,-20,1173,1093,942006,16,275.3,13.000000,10
3,202101010101,2018101447,2021-06-12,0,0,0.0,1200,2,0,-3.0,...,56.0,498,-4,1091,1076,546800,11,57.7,11.000000,12
4,202101010101,2018101539,2021-06-12,0,0,0.0,1200,2,0,-3.0,...,54.0,456,8,1015,417,163002,4,7.5,3.666667,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235833,202510020812,2022105755,2025-07-20,9,0,0.0,1200,0,0,2.0,...,53.0,452,4,1187,1135,520031,3,7.2,6.000000,5
235834,202510020812,2017101767,2025-07-20,9,0,0.0,1200,0,0,2.0,...,58.0,480,8,1125,1120,892002,16,111.0,7.666667,12
235835,202510020812,2020106727,2025-07-20,9,0,0.0,1200,0,0,2.0,...,58.0,544,0,1213,1205,798033,5,18.1,8.333333,8
235836,202510020812,2021103312,2025-07-20,9,0,0.0,1200,0,0,2.0,...,58.0,496,-8,1154,1157,170800,8,29.0,4.000000,3


In [4]:
import pandas as pd
import numpy as np

df = df_train.copy()
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 235838 entries, 0 to 235837
Data columns (total 24 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   race_id         235838 non-null  int64         
 1   horse_id        235838 non-null  int64         
 2   race_date       235838 non-null  datetime64[us]
 3   place           235838 non-null  int64         
 4   race_type       235838 non-null  int64         
 5   around          229062 non-null  float64       
 6   course_len      235838 non-null  int64         
 7   weather         235838 non-null  int64         
 8   ground_state    235838 non-null  int64         
 9   race_class      235838 non-null  float64       
 10  wakuban         235838 non-null  Int64         
 11  umaban          235838 non-null  Int64         
 12  sex             235838 non-null  int64         
 13  age             235838 non-null  Int64         
 14  impost          235838 non-null  float64       

In [5]:
df.columns

Index(['race_id', 'horse_id', 'race_date', 'place', 'race_type', 'around',
       'course_len', 'weather', 'ground_state', 'race_class', 'wakuban',
       'umaban', 'sex', 'age', 'impost', 'weight', 'weight_diff', 'jockey_id',
       'trainer_id', 'owner_id', 'popularity', 'tansho_odds', 'hr_rank_mean_3',
       'rank'],
      dtype='str')

In [6]:
df.dtypes

race_id                    int64
horse_id                   int64
race_date         datetime64[us]
place                      int64
race_type                  int64
around                   float64
course_len                 int64
weather                    int64
ground_state               int64
race_class               float64
wakuban                    Int64
umaban                     Int64
sex                        int64
age                        Int64
impost                   float64
weight                     int64
weight_diff                int64
jockey_id                  int64
trainer_id                 int64
owner_id                  object
popularity                 Int64
tansho_odds              float64
hr_rank_mean_3           float64
rank                       int64
dtype: object

In [7]:
dup = df.duplicated(["race_id", "horse_id"], keep=False)
dup.sum(), df.loc[dup, ["race_id","horse_id"]].head(20)

(np.int64(0),
 Empty DataFrame
 Columns: [race_id, horse_id]
 Index: [])

In [8]:
g = df.groupby("race_id", sort=False)

qc = pd.DataFrame({
    "n_horses": g.size(),
    "race_date_nunique": g["race_date"].nunique(),
    "place_nunique": g["place"].nunique(),
    "race_type_nunique": g["race_type"].nunique(),
    "course_len_nunique": g["course_len"].nunique(),
    "ground_state_nunique": g["ground_state"].nunique(),
    "weather_nunique": g["weather"].nunique(),
    "around_nunique": g["around"].nunique(dropna=False),  # 欠損含めて確認
    "umaban_min": g["umaban"].min(),
    "umaban_max": g["umaban"].max(),
    "rank_min": g["rank"].min(),
    "rank_max": g["rank"].max(),
})

# 異常フラグ
qc["umaban_out_of_range"] = (qc["umaban_min"] < 1) | (qc["umaban_max"] > qc["n_horses"])
qc["rank_out_of_range"] = (qc["rank_min"] < 1) | (qc["rank_max"] > qc["n_horses"])

qc.sort_values(
    ["race_date_nunique","place_nunique","race_type_nunique","course_len_nunique",
     "ground_state_nunique","weather_nunique","umaban_out_of_range","rank_out_of_range","n_horses"],
    ascending=False
).head(50)


,n_horses,race_date_nunique,place_nunique,race_type_nunique,course_len_nunique,ground_state_nunique,weather_nunique,around_nunique,umaban_min,umaban_max,rank_min,rank_max,umaban_out_of_range,rank_out_of_range
race_id,,,,,,,,,,,,,,
202304040405,14,1,1,1,1,1,1,1,1,15,1,15,True,True
202207011105,12,1,1,1,1,1,1,1,1,13,1,13,True,True
202104030206,17,1,1,1,1,1,1,1,1,18,1,17,True,False
202104030302,17,1,1,1,1,1,1,1,1,18,1,17,True,False
202104030406,17,1,1,1,1,1,1,1,1,18,1,17,True,False
202104040211,17,1,1,1,1,1,1,1,1,18,1,17,True,False
202104040305,17,1,1,1,1,1,1,1,1,18,1,17,True,False
202104050112,17,1,1,1,1,1,1,1,2,18,1,17,True,False
202104050202,17,1,1,1,1,1,1,1,1,18,1,17,True,False
